# Análisis Comentarios

In [2]:
# Importamos las librerías

import nltk
import pandas as pd
from datetime import datetime
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from nltk.tokenize import RegexpTokenizer

In [4]:
df_coments = pd.read_csv("instagram_coments_lomeli.csv", usecols=["_aacl"])
df_coments = df_coments.dropna()
df_coments = df_coments.rename(columns={"_aacl": "texto"})
df_coments

,texto
2,"En Jalisco, Morena no está para improvisar, no..."
6,Por la coordinación de la defensa de la en
9,La situación de Carlos Lomelí la resolverá el ...
12,En nuestra mañanera dimos voz a un grupo de ve...
15,Que esta en juego este 4 de Junio en las elecc...
18,Me siento muy afortunado de conocer y trabajar...
21,Creo que en jalisco el 1 de julio tendra gober...
24,Una fiesta cívica el cierre de campaña en Zapo...
27,Prosperando para un mejor Zapopan 💪🏻
30,Yo y mis gestos de atención en la apertura de ...


In [5]:
df_coments['texto'] = df_coments['texto'].str.lower()
tokenizer = RegexpTokenizer(r'\w+')
df_coments['tokens'] = df_coments['texto'].apply(tokenizer.tokenize)
df_coments.to_csv("coments_insta.csv")
df_coments

,texto,tokens
2,"en jalisco, morena no está para improvisar, no...","[en, jalisco, morena, no, está, para, improvis..."
6,por la coordinación de la defensa de la en,"[por, la, coordinación, de, la, defensa, de, l..."
9,la situación de carlos lomelí la resolverá el ...,"[la, situación, de, carlos, lomelí, la, resolv..."
12,en nuestra mañanera dimos voz a un grupo de ve...,"[en, nuestra, mañanera, dimos, voz, a, un, gru..."
15,que esta en juego este 4 de junio en las elecc...,"[que, esta, en, juego, este, 4, de, junio, en,..."
18,me siento muy afortunado de conocer y trabajar...,"[me, siento, muy, afortunado, de, conocer, y, ..."
21,creo que en jalisco el 1 de julio tendra gober...,"[creo, que, en, jalisco, el, 1, de, julio, ten..."
24,una fiesta cívica el cierre de campaña en zapo...,"[una, fiesta, cívica, el, cierre, de, campaña,..."
27,prosperando para un mejor zapopan 💪🏻,"[prosperando, para, un, mejor, zapopan]"
30,yo y mis gestos de atención en la apertura de ...,"[yo, y, mis, gestos, de, atención, en, la, ape..."
